### Identifica diferenças geométrias e temáticas entre várias ESRI Shapefiles ##

In [ ]:
from gasp.anls.ovlay import check_shape_diff

# Inputs

SHAPES_TO_COMPARE = {
    '/home/osmtolulc/webvia/coimbra/v11_cmb.shp'       : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v12_cmb.shp'       : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v13_cmb_2x2.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v13_cmb_5x5.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v13_cmb_10x10.shp' : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v14_cmb_2x2.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v14_cmb_5x5.shp'   : "cls_int",
    '/home/osmtolulc/webvia/coimbra/v14_cmb_10x10.shp' : "cls_int"
}

OUT_FOLDER = '/home/osmtolulc/webvia/cmb_anls'
REPORT     = '/home/osmtolulc/webvia/cmb_compare.xlsx'

conPARAM = {
    "HOST" : "localhost", "PORT" : "5432",
    "USER" : "postgres", "PASSWORD" : "admin", "TEMPLATE" : "template_postgis"
}

DB = "cmb_compare"

srs_code = 3857

RASTER_TEMPLATE = '/home/osmtolulc/webvia/boundaries/coimbra_20x20.shp'

check_shape_diff(SHAPES_TO_COMPARE, OUT_FOLDER, REPORT, conPARAM, DB, srs_code,
                GIS_SOFTWARE="GRASS", GRASS_REGION_TEMPLATE=RASTER_TEMPLATE)

/usr/local/lib/python2.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Union between /home/osmtolulc/webvia/cmb_anls/v14_cmb_5x5_rn.shp and /home/osmtolulc/webvia/cmb_anls/v11_cmb_rn.shp
0:04:24
Union between /home/osmtolulc/webvia/cmb_anls/v14_cmb_5x5_rn.shp and /home/osmtolulc/webvia/cmb_anls/v12_cmb_rn.shp
0:04:20
Union between /home/osmtolulc/webvia/cmb_anls/v14_cmb_5x5_rn.shp and /home/osmtolulc/webvia/cmb_anls/v13_cmb_2x2_rn.shp


### Union Analysis ###

In [ ]:
from gasp.anls.ovlay import union

LYR_A  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v13_coimbra.shp'
LYR_B  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_coimbra_clp.shp'
outSHP = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\teste_v13_v11.shp'
API    = 'saga'

union(LYR_A, LYR_B, outSHP, api_gis=API)

### Union Analysis in GRASS GIS ###

In [ ]:
from gasp.cpu.grs import run_grass
from gasp.pyt.oss import fprop

GRS_WORKSPACE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19'
LYR_A  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v13_coimbra.shp'
LYR_B  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_coimbra_clp.shp'
outSHP = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\grass_v13_v11.shp'

grsbase = run_grass(GRS_WORKSPACE, location='tst_union', srs=3763)
import grass.script.setup as gsetup
gsetup.init(grsbase, GRS_WORKSPACE, 'tst_union', 'PERMANENT')

# Import data
from gasp.to.shp.grs import shp_to_grs

lyr_a = shp_to_grs(LYR_A, fprop(LYR_A, 'fn'), asCMD=True)
lyr_b = shp_to_grs(LYR_B, fprop(LYR_B, 'fn'), asCMD=True)

import datetime
time_a = datetime.datetime.now().replace(microsecond=0)

from gasp.anls.ovlay import union
shpUnion = union(lyr_a, lyr_b, fprop(outSHP, 'fn'), api_gis="grass_cmd")

time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

# Export data
from gasp.to.shp.grs import grs_to_shp
result = grs_to_shp(shpUnion, outSHP, "area")

In [ ]:
"""
Optimized Union Analysis
"""

import os
import datetime
from gasp.pyt.oss import fprop
from gasp.cpu.gdl.mng.sample import create_fishnet
from gasp.mng.feat  import eachfeat_to_newshp
from gasp.cpu.grs   import run_grass
from gasp.anls.exct import split_shp_by_attr

LYR_A        = '/home/osmtolulc/results_fev19/v11_cmb.shp'
LYR_A_COL    = 'cls_int'
LYR_B        = '/home/osmtolulc/results_fev19/v12_cmb.shp'
LYR_B_COL    = 'cls_int'
OUT_FOLDER   = '/home/osmtolulc/tst_union'
REF_BOUNDARY = '/home/osmtolulc/boundaries/coimbra_20x20.shp'
EPSG = 3857

if not os.path.exists(OUT_FOLDER):
    from gasp.oss.ops import mkdir
    
    mkdir(OUT_FOLDER)

# Create Fishnet
gridShp = create_fishnet(
    REF_BOUNDARY, os.path.join(OUT_FOLDER, 'ref_grid.shp'),
    rowN=4, colN=4
)

# Split Fishnet in several files
cellsShp = eachfeat_to_newshp(gridShp, OUT_FOLDER, epsg=EPSG)

# INIT GRASS GIS Session
grsbase = run_grass(OUT_FOLDER, location="proc", srs=REF_BOUNDARY)
import grass.script.setup as gsetup
gsetup.init(grsbase, OUT_FOLDER, "proc", 'PERMANENT')

# Add data to GRASS GIS
from gasp.to.shp.grs import shp_to_grs
cellsShp   = [shp_to_grs(
    shp, fprop(shp, 'fn'), asCMD=True
) for shp in cellsShp]

LYR_A = shp_to_grs(LYR_A, fprop(LYR_A, 'fn'), asCMD=True)
LYR_B = shp_to_grs(LYR_B, fprop(LYR_B, 'fn'), asCMD=True)

# Clip Layers A and B for each CELL in fishnet
from gasp.anls.ovlay import clip

time_a = datetime.datetime.now().replace(microsecond=0)

LYRS_A = [clip(
    fprop(LYR_A, 'fn'), fprop(cellsShp[x], 'fn'),
    fprop(LYR_A, 'fn') + "_" + str(x), api_gis="grass_cmd"
) for x in range(len(cellsShp))]
time_b = datetime.datetime.now().replace(microsecond=0)

LYRS_B = [clip(
    fprop(LYR_B, 'fn'), fprop(cellsShp[x], 'fn'),
    fprop(LYR_B, 'fn') + "_" + str(x), api_gis="grass_cmd"
) for x in range(len(cellsShp))]

time_c = datetime.datetime.now().replace(microsecond=0)

print "Total Time:"
print time_c - time_a
print "LYR_A_CLIP"
print time_b - time_a
print "LYR_B_CLIP"
print time_c - time_b

time_d = datetime.datetime.now().replace(microsecond=0)
from gasp.anls.ovlay import union_for_all_pairs as union
UNION_SHP = union([(LYRS_A[i], LYRS_B[i]) for i in range(len(LYRS_A))])
time_e = datetime.datetime.now().replace(microsecond=0)
print time_e - time_d

from gasp.to.shp.grs import grs_to_shp

_UNION_SHP = [grs_to_shp(
    shp, os.path.join(OUT_FOLDER, shp + ".shp"), "area"
) for shp in UNION_SHP]

from gasp.mng.gen import merge_feat
FSHP = [os.path.join('/home/osmtolulc/tst_union/uniao_{}.shp').format(x) for x in range(16)]
MERGED_SHP = merge_feat(FSHP, '/home/osmtolulc/uniao_tst.shp')

In [ ]:
"""
Optimized Union Analysis with MultiProcess
"""

import os
import datetime
from gasp.oss import fprop
from gasp.cpu.gdl.mng.sample import create_fishnet
from gasp.mng.feat  import eachfeat_to_newshp
from gasp.cpu.grs   import run_grass
from gasp.anls.exct import split_shp_by_attr

LYR_A        = '/home/osmtolulc/results_fev19/v11_cmb.shp'
LYR_A_COL    = 'cls_int'
LYR_B        = '/home/osmtolulc/results_fev19/v13_cmb_2x2.shp'
LYR_B_COL    = 'cls_int'
OUT_FOLDER   = '/home/osmtolulc/tst_union'
REF_BOUNDARY = '/home/osmtolulc/boundaries/coimbra_20x20.shp'
RESULT_UNION = '/home/osmtolulc/uniao_v11_v13.shp'
EPSG = 3857

if not os.path.exists(OUT_FOLDER):
    from gasp.oss.ops import mkdir
    
    mkdir(OUT_FOLDER)

# Create Fishnet
gridShp = create_fishnet(
    REF_BOUNDARY, os.path.join(OUT_FOLDER, 'ref_grid.shp'),
    rowN=4, colN=4
)

# Split Fishnet in several files
cellsShp = eachfeat_to_newshp(gridShp, OUT_FOLDER, epsg=EPSG)

def clip_and_union(lyr_a, lyr_b, cell_shp, work, ref, proc, output):
    # Start GRASS GIS Session
    grsbase = run_grass(work, location="proc_" + str(proc), srs=ref)
    import grass.script.setup as gsetup
    gsetup.init(grsbase, work, "proc_" + str(proc), 'PERMANENT')
    
    # Import GRASS GIS modules
    from gasp.to.shp.grs import shp_to_grs
    from gasp.anls.ovlay import union
    from gasp.anls.ovlay import clip
    from gasp.to.shp.grs import grs_to_shp
    
    # Add data to GRASS
    a = shp_to_grs(LYR_A, fprop(LYR_A, 'fn'), asCMD=True)
    b = shp_to_grs(LYR_B, fprop(LYR_B, 'fn'), asCMD=True)
    c = shp_to_grs(cell_shp, fprop(cell_shp, 'fn'), asCMD=True)
    
    # Clip
    a_clip = clip(a, c, "{}_clip".format(a), api_gis="grass_cmd")
    b_clip = clip(b, c, "{}_clip".format(b), api_gis="grass_cmd")
    
    # Union
    u_shp = union(a_clip, b_clip, "un_{}".format(c), api_gis="grass_cmd")
    
    # Export
    o = grs_to_shp(u_shp, output, "area")

import multiprocessing

time_a = datetime.datetime.now().replace(microsecond=0)

thrds = [multiprocessing.Process(
    target=clip_and_union, name="th-{}".format(i), args=(
        LYR_A, LYR_B, cellsShp[i],
        os.path.join(OUT_FOLDER, "th_{}".format(i)),
        REF_BOUNDARY, i, os.path.join(OUT_FOLDER, "uniao_{}.shp".format(i))
    )
) for i in range(len(cellsShp))]

for t in thrds:
    t.start()

for t in thrds:
    t.join()

time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

from gasp.mng.gen import merge_feat
FSHP = [os.path.join(OUT_FOLDER, "uniao_{}.shp".format(i)) for x in range(len(cellsShp))]
MERGED_SHP = merge_feat(FSHP, RESULT_UNION)

In [ ]:
from gasp.anls.ovlay import optimized_union_anls

import datetime

time_a = datetime.datetime.now().replace(microsecond=0)
optimized_union_anls(
    "/home/osmtolulc/webvia/coimbra/v11_cmb.shp",
    "/home/osmtolulc/webvia/coimbra/v13_cmb_2x2.shp",
    "/home/osmtolulc/webvia/union_v11_v13_regular.shp",
    "/home/osmtolulc/webvia/boundaries/coimbra_20x20.shp", 3857, 
    multiProcess=None
)
time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a